# QCEW Summarization Data Kit

## Introduction

This Jupyter Notebook is designed to summarize Quarterly Census of Employment and Wages (QCEW) datasets, retreived from [morpc-qcew-data-kit](https://github.com/olivergwynn/qcew_data_kit/tree/main) . The dataset(s) processed by this notebook contain detailed employment statistics, as defined by the [technical documentation](https://www.bls.gov/cew/additional-resources/open-data/csv-data-slices.htm), including the number of establishments, employment levels, total quarterly wages, and more, broken down by industry and ownership sectors for each county and a whole regional summary. This script takes wide/long-form QCEW data from counties and adds regional summary data.

## Process Outline

The process carried out by this workflow can be described as follows:
  - Wide-form standardized QCEW files are summarized to generate wide-form annual and/or quarterly tables with county and region-wide data.
  - Newly created wide-form tables are proccessed to generate long-form counterparts.
  - For each processed long and wide-from '.csv', a .resource.yaml file is created, following the Frictionless Data Resource specification. This file includes metadata about the CSV file, such as its name, path, format, and the schema it conforms to, as well as a hash code for integrity checking. Additionally, it contains descriptive information about the dataset and references to its source
  - The YAML files for schemas and resource descriptors are used to make data more usable by simplifying its publication and consumption. By adhering to Frictionless standards, the script ensures that the datasets it produces are easily shareable, validatable, and integrable into a wide range of data tools and platforms.

## Setup

### Import required packages

In [1]:
import os
import numpy as np
import pandas as pd
import yaml
import json
import frictionless
import requests
import sys
from io import StringIO
sys.path.append(os.path.normpath("../../morpc-common"))
import morpc

### Parameters

#### Static parameters

In [2]:
# Location where output files will be saved
OUTPUT_DIR = os.path.normpath("./output_data")

# Location where input files must be placed
INPUT_DIR = os.path.normpath("./input_data")

# List of identifying columns for long-form tables
id_vars = [
    'area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code', 'year', 'qtr', 'disclosure_code', 'lq_disclosure_code', 'oty_disclosure_code'
]

# File name for long-form quarterly table
QCEW_QUARTERLY_LONG_OUTPUT_NAME  = "qcew_quarterly_long_summarized.csv" 
# File name for wide-form quarterly table
QCEW_QUARTERLY_WIDE_OUTPUT_NAME  = "qcew_quarterly_wide_summarized.csv" 

# File name for long-form annual table
QCEW_ANNUAL_LONG_OUTPUT_NAME  = "qcew_annual_long_summarized.csv" 
# File name for wide-form annual table
QCEW_ANNUAL_WIDE_OUTPUT_NAME  = "qcew_annual_wide_summarized.csv" 

# Quarterly data paths
QCEW_QUARTERLY_LONG_OUTPUT_PATH = os.path.join(OUTPUT_DIR, QCEW_QUARTERLY_LONG_OUTPUT_NAME)
QCEW_QUARTERLY_WIDE_OUTPUT_PATH = os.path.join(OUTPUT_DIR, QCEW_QUARTERLY_WIDE_OUTPUT_NAME)

QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE = "qcew_quarterly_long_summarized.resource.yaml" 
QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE_PATH = os.path.join(OUTPUT_DIR, QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE)
QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE = "qcew_quarterly_wide_summarized.resource.yaml"
QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE_PATH = os.path.join(OUTPUT_DIR, QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE)


# Annual paths
QCEW_ANNUAL_LONG_OUTPUT_PATH = os.path.join(OUTPUT_DIR, QCEW_ANNUAL_LONG_OUTPUT_NAME)
QCEW_ANNUAL_WIDE_OUTPUT_PATH = os.path.join(OUTPUT_DIR, QCEW_ANNUAL_WIDE_OUTPUT_NAME)

QCEW_ANNUAL_LONG_OUTPUT_RESOURCE = "qcew_annual_long_summarized.resource.yaml" 
QCEW_ANNUAL_LONG_OUTPUT_RESOURCE_PATH = os.path.join(OUTPUT_DIR, QCEW_ANNUAL_LONG_OUTPUT_RESOURCE)
QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE = "qcew_annual_wide_summarized.resource.yaml"
QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH = os.path.join(OUTPUT_DIR, QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE)


# Define quarterly and annual schema directories from local copies
QUARTERLY_TABLE_SCHEMA_FILENAME = "morpc-qcew-quarterly.schema.yaml"
QUARTERLY_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, QUARTERLY_TABLE_SCHEMA_FILENAME)
ANNUAL_TABLE_SCHEMA_FILENAME = "morpc-qcew-annual.schema.yaml"
ANNUAL_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, ANNUAL_TABLE_SCHEMA_FILENAME)
LONG_TABLE_SCHEMA_FILENAME = "morpc-qcew-long.schema.yaml"
LONG_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, LONG_TABLE_SCHEMA_FILENAME)


QCEW_ANNUAL_WIDE_INPUT_NAME  = "qcew_annual_wide.csv" 

QCEW_ANNUAL_WIDE_INPUT_PATH = os.path.join(INPUT_DIR, QCEW_ANNUAL_WIDE_INPUT_NAME)

QCEW_QUARTERLY_WIDE_INPUT_NAME  = "qcew_quarterly_wide.csv" 

QCEW_QUARTERLY_WIDE_INPUT_PATH = os.path.join(INPUT_DIR, QCEW_QUARTERLY_WIDE_INPUT_NAME)

#### User-set parameters

### Define inputs

In [3]:
print("Annual schema file stored in: {}".format(ANNUAL_TABLE_SCHEMA_PATH))
print("Quarterly schema file stored in: {}".format(QUARTERLY_TABLE_SCHEMA_PATH))
print("Long schema file stored in: {}".format(LONG_TABLE_SCHEMA_PATH))
print("Standardized quarterly wide-form QCEW data to be summarized must be stored in: {}".format(QCEW_QUARTERLY_WIDE_INPUT_PATH))
print("Standardized annual wide-form QCEW data to be summarized must be stored in: {}".format(QCEW_ANNUAL_WIDE_INPUT_PATH))

Annual schema file stored in: output_data\morpc-qcew-annual.schema.yaml
Quarterly schema file stored in: output_data\morpc-qcew-quarterly.schema.yaml
Long schema file stored in: output_data\morpc-qcew-long.schema.yaml
Standardized quarterly wide-form QCEW data to be summarized must be stored in: input_data\qcew_quarterly_wide.csv
Standardized annual wide-form QCEW data to be summarized must be stored in: input_data\qcew_annual_wide.csv


### Define outputs

In [4]:
print("Summarized Long quarterly QCEW data will be saved to: {}".format(QCEW_QUARTERLY_LONG_OUTPUT_PATH))
print("Summarized long quarterly QCEW data resource files will be saved to: {}".format(QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE_PATH))
print("Summarized wide quarterly QCEW data will be saved to: {}".format(QCEW_QUARTERLY_WIDE_OUTPUT_PATH))
print("Summarized wide quarterly QCEW data resource files will be saved to: {}".format(QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE_PATH))
print("")
print("Summarized long annual QCEW data will be saved to: {}".format(QCEW_ANNUAL_LONG_OUTPUT_PATH))
print("Summarized long annual QCEW data resource files will be saved to: {}".format(QCEW_ANNUAL_LONG_OUTPUT_RESOURCE_PATH))
print("Summarized wide annual QCEW data will be saved to: {}".format(QCEW_ANNUAL_WIDE_OUTPUT_PATH))
print("Summarized wide annual QCEW data resource files will be saved to: {}".format(QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH))

Summarized Long quarterly QCEW data will be saved to: output_data\qcew_quarterly_long_summarized.csv
Summarized long quarterly QCEW data resource files will be saved to: output_data\qcew_quarterly_long_summarized.resource.yaml
Summarized wide quarterly QCEW data will be saved to: output_data\qcew_quarterly_wide_summarized.csv
Summarized wide quarterly QCEW data resource files will be saved to: output_data\qcew_quarterly_wide_summarized.resource.yaml

Summarized long annual QCEW data will be saved to: output_data\qcew_annual_long_summarized.csv
Summarized long annual QCEW data resource files will be saved to: output_data\qcew_annual_long_summarized.resource.yaml
Summarized wide annual QCEW data will be saved to: output_data\qcew_annual_wide_summarized.csv
Summarized wide annual QCEW data resource files will be saved to: output_data\qcew_annual_wide_summarized.resource.yaml


## Doing Things!

### Wide Annual Summary

In [5]:
def QCEWAnnualAverage(data):
    # Ensure that numeric columns are of numeric types before any operations
    numeric_columns = ['annual_avg_estabs', 'annual_avg_emplvl', 'total_annual_wages',
                       'taxable_annual_wages', 'annual_contributions']
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Calculate additional metrics for the original data
    data['avg_annual_pay'] = (data['total_annual_wages'] /
                              data['annual_avg_emplvl']).replace([np.inf, -np.inf], np.nan)
    data['annual_avg_wkly_wage'] = data['avg_annual_pay'] / 52.1775

    # Create a copy of the data to perform the groupby operation and create summarized rows
    summarized_data = data.copy()

    # Perform the groupby operation to create summarized rows
    group_columns = ["own_code", "industry_code", "agglvl_code", "year", "qtr"]
    summarized_data = summarized_data.groupby(group_columns).agg({col: 'sum' for col in numeric_columns}).reset_index()

    # Add the 'area_fips' as '00000' for summarized rows
    summarized_data['area_fips'] = '00000'

    # Calculate additional metrics for the summarized data
    summarized_data['avg_annual_pay'] = (summarized_data['total_annual_wages'] /
                                         summarized_data['annual_avg_emplvl']).replace([np.inf, -np.inf], np.nan)
    summarized_data['annual_avg_wkly_wage'] = summarized_data['avg_annual_pay'] / 52.1775

    # Append the summarized rows to the original data
    data_grouped = pd.concat([data, summarized_data], ignore_index=True)

    # Perform year-over-year change calculations on the combined dataset
    for metric in numeric_columns + ['avg_annual_pay', 'annual_avg_wkly_wage']:
        data_grouped[f'oty_{metric}_chg'] = data_grouped.groupby(group_columns)[metric].diff()
        data_grouped[f'oty_{metric}_pct_chg'] = (
            data_grouped[f'oty_{metric}_chg'] / 
            data_grouped.groupby(group_columns)[metric].shift(1)
        ) * 100

    # Replace infinite values or NaN that might have been generated due to division by zero
    data_grouped.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_grouped = data_grouped.infer_objects(copy=False)

    # Fill NaN values that result from the diff operation on the first entry of each group with zero
    data_grouped.fillna(0, inplace=True)

    # Return the combined data with original rows and their corresponding summarized rows
    return data_grouped

def own_coder(df):
    # Define columns to group by and columns to sum
    group_columns = ['year', 'qtr', 'area_fips', 'industry_code', 'agglvl_code']
    sum_columns = ['annual_avg_estabs', 'annual_avg_emplvl', 'total_annual_wages',
                   'taxable_annual_wages', 'annual_contributions']

    # Group by the specified columns and sum the other columns
    grouped = df.groupby(group_columns)[sum_columns].sum().reset_index()

    # Assign the new 'own_code'
    grouped['own_code'] = 10

    # Ensure the new aggregated rows include all necessary columns
    # by adding missing columns with default or NaN values
    for column in df.columns:
        if column not in grouped:
            grouped[column] = None  # or pd.NA for pandas' NA type

    # Exclude empty or all-NA columns
    grouped = grouped.dropna(axis=1, how='all')

    # Concatenate the original DataFrame with the new aggregated DataFrame
    result_df = pd.concat([df, grouped], ignore_index=True)

    return result_df

def get_numeric_columns():
    # Return a list of all the columns you expect to be numeric
    return [
        'annual_avg_estabs', 'annual_avg_emplvl', 'total_annual_wages',
        'taxable_annual_wages', 'annual_contributions', 'avg_annual_pay', 'annual_avg_wkly_wage'
        # Add more columns as needed
    ]

def QCEWAnnualSummary(input_csv_path, output_csv_path):
    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        raise Exception(f"Error loading input CSV: {e}")

    # Ensure 'qtr' and other numeric columns are processed correctly
    columns_to_process = get_numeric_columns()

    df = own_coder(df)
    df = QCEWAnnualAverage(df)
    
    # Group by 'area_fips', 'year', 'qtr' and calculate proportions 
    grouped_df = df.groupby(['area_fips', 'year', 'qtr'])
    for name, group in grouped_df:
        divisor_row = group[(group['own_code'] == 10) & (group['agglvl_code'] == 70)]
        if not divisor_row.empty:
            divisor_values = divisor_row.iloc[0][columns_to_process]

    df = df.replace(',', '', regex=True)
    # Save the modified DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

# Load the annual wide CSV
df_annual = pd.read_csv(QCEW_ANNUAL_WIDE_INPUT_PATH)

# Verify if there's data in the annual DataFrame
if not df_annual.empty:
    QCEWAnnualSummary(QCEW_ANNUAL_WIDE_INPUT_PATH, QCEW_ANNUAL_WIDE_OUTPUT_PATH)
else: 
    print("No annual data found in: {}".format(QCEW_ANNUAL_WIDE_INPUT_PATH))


### Wide Quarterly Summary

In [6]:
import pandas as pd
import numpy as np

def QCEWAnnualAverage(data):
    # Ensure that numeric columns are of numeric types before any operations
    numeric_columns = ['qtrly_estabs','month1_emplvl','month2_emplvl','month3_emplvl','total_qtrly_wages','taxable_qtrly_wages','qtrly_contributions']
    
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Calculate additional metrics for the original data
    data['avg_wkly_wage'] = (data['total_qtrly_wages'] /
                              ((data['month1_emplvl'] + data['month2_emplvl'] + data['month3_emplvl'])/3) /13    ).replace([np.inf, -np.inf], np.nan)

    # Create a copy of the data to perform the groupby operation and create summarized rows
    summarized_data = data.copy()

    # Perform the groupby operation to create summarized rows
    group_columns = ["own_code", "industry_code", "agglvl_code", "year", "qtr"]
    summarized_data = summarized_data.groupby(group_columns).agg({col: 'sum' for col in numeric_columns}).reset_index()

    # Calculate additional metrics for the summarized data
    summarized_data['avg_wkly_wage'] = (summarized_data['total_qtrly_wages'] /
                              ((summarized_data['month1_emplvl'] + summarized_data['month2_emplvl'] + summarized_data['month3_emplvl'])/3) /13    ).replace([np.inf, -np.inf], np.nan)

    summarized_data["area_fips"]='0'

    # Append the summarized rows to the original data
    data_grouped = pd.concat([data, summarized_data], ignore_index=True)

    # Perform year-over-year change calculations on the combined dataset
    for metric in numeric_columns + ['avg_wkly_wage']:
        data_grouped[f'oty_{metric}_chg'] = data_grouped.groupby(group_columns)[metric].diff()
        data_grouped[f'oty_{metric}_pct_chg'] = (
            data_grouped[f'oty_{metric}_chg'] / 
            data_grouped.groupby(group_columns)[metric].shift(1)
        ) * 100

    # Replace infinite values or NaN that might have been generated due to division by zero
    data_grouped.replace([np.inf, -np.inf], np.nan, inplace=True)
    data_grouped = data_grouped.infer_objects(copy=False)

    # Fill NaN values that result from the diff operation on the first entry of each group with zero
    data_grouped.fillna(0, inplace=True)

    return data_grouped

def own_coder(df):
    # Define columns to group by and columns to sum
    group_columns = ['year', 'qtr', 'area_fips', 'industry_code', 'agglvl_code']
    sum_columns = ['qtrly_estabs', 'month1_emplvl', 'month2_emplvl',
                   'month3_emplvl', 'total_qtrly_wages', 'taxable_qtrly_wages','qtrly_contributions','avg_wkly_wage']

    # Group by the specified columns and sum the other columns
    grouped = df.groupby(group_columns)[sum_columns].sum().reset_index()

    # Assign the new 'own_code'
    grouped['own_code'] = 10

    # Ensure the new aggregated rows include all necessary columns
    # by adding missing columns with default or NaN values
    for column in df.columns:
        if column not in grouped:
            grouped[column] = None  # or pd.NA for pandas' NA type

    # Exclude empty or all-NA columns
    grouped = grouped.dropna(axis=1, how='all')

    # Concatenate the original DataFrame with the new aggregated DataFrame
    result_df = pd.concat([df, grouped], ignore_index=True)

    return result_df

def QCEWQuarterlySummary(input_csv_path, output_csv_path):
    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        raise Exception(f"Error loading input CSV: {e}")

    columns_to_process = ['qtrly_estabs', 'month1_emplvl', 'month2_emplvl',
                   'month3_emplvl', 'total_qtrly_wages', 'taxable_qtrly_wages','qtrly_contributions','avg_wkly_wage']

    df = own_coder(df)
    df = QCEWAnnualAverage(df)
    
    # Group by 'area_fips', 'year', 'qtr' and calculate proportions 
    grouped_df = df.groupby(['area_fips', 'year', 'qtr'])
    for name, group in grouped_df:
        divisor_row = group[(group['own_code'] == 10) & (group['agglvl_code'] == 70)]
        if not divisor_row.empty:
            divisor_values = divisor_row.iloc[0][columns_to_process]

    df = df.replace(',', '', regex=True)
    # Save the modified DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

# Load the quarterly wide CSV
df_quarterly = pd.read_csv(QCEW_QUARTERLY_WIDE_INPUT_PATH)

# Verify if there's data in the quarterly DataFrame
if not df_quarterly.empty:
    QCEWQuarterlySummary(QCEW_QUARTERLY_WIDE_INPUT_PATH, QCEW_QUARTERLY_WIDE_OUTPUT_PATH)
else: 
    print("No quarterly data found in: {}".format(QCEW_QUARTERLY_WIDE_INPUT_PATH))


### Creating long-form annual table, if wide-form annual data exists

Melt the summarized wide-form annual data into a long-form table where each row corresponds to a single variable for a given county, year, establishment ownership, establishment size, industry, aggregation code, and disclosure codes

In [7]:
annualDtypes = {
    'disclosure_code': 'str',
    'industry_code': 'str',
    'lq_disclosure_code': 'str',
    'oty_disclosure_code': 'str'
}

# Load the annual wide CSV
df_annual = pd.read_csv(QCEW_ANNUAL_WIDE_OUTPUT_PATH, dtype=annualDtypes)

# Verify if there's data in the annual DataFrame
if not df_annual.empty:
    value_vars_annual = df_annual.columns.difference(id_vars)
    df_annual_long = pd.melt(df_annual, id_vars=id_vars, value_vars=value_vars_annual, 
                             var_name='variable', value_name='value')
    df_annual_long.to_csv(QCEW_ANNUAL_LONG_OUTPUT_PATH, index=False)
    (f"Annual QCEW data have been saved as long into: {QCEW_ANNUAL_LONG_OUTPUT_PATH}")
else: 
    print("No annual data to melt")

### Creating long-form quarterly table, if wide-form quarterly data exists

Melt the summarized wide-form quarterly data into a long-form table where each row corresponds to a single variable for a given county, year, quarter, establishment ownership, establishment size, industry, aggregation code, and disclosure codes

In [8]:
quarterlyDtypes = {
    'disclosure_code': 'str',
    'industry_code': 'str',
    'lq_disclosure_code': 'str',
    'oty_disclosure_code': 'str'
    #'lq_avg_wkly_wage': 'float64',
}

# Load the annual wide CSV
df_annual = pd.read_csv(QCEW_QUARTERLY_WIDE_OUTPUT_PATH, dtype=quarterlyDtypes)

# Verify if there's data in the annual DataFrame
if not df_annual.empty:
    value_vars_quarterly = df_annual.columns.difference(id_vars)
    df_quarterly_long = pd.melt(df_annual, id_vars=id_vars, value_vars=value_vars_quarterly, 
                             var_name='variable', value_name='value')
    df_quarterly_long.to_csv(QCEW_QUARTERLY_LONG_OUTPUT_PATH, index=False)
    (f"Quarterly QCEW data have been saved as long into: {QCEW_QUARTERLY_LONG_OUTPUT_PATH}")
else: 
    print("No annual data to melt")

## Create and validate resource file for annual wide-form table, if it exists

In [9]:
df_wide_annual = pd.read_csv(QCEW_ANNUAL_WIDE_OUTPUT_PATH, dtype=annualDtypes)

# Finding the maximum and minimum values in the 'year' column
max_year = df_wide_annual['year'].max()
min_year = df_wide_annual['year'].min()

# Update title and description with the county name
title = f"Compiled QCEW County Data, Annual, {min_year}-{max_year} (wide form)"
description = f"Employment and wage data for counties in MOPRC region, derived from the U.S. Bureau of Labor Statistics."

# Resource creation for WIDE ANNUAL
if not df_wide_annual.empty:
    acsResource = {
        "name": "qcew_annual_wide",
        "title": title,
        "description": description,
        "path": QCEW_ANNUAL_WIDE_OUTPUT_NAME,
        "format": "csv",
        "mediatype": "text/csv",
        "encoding": "utf-8",
        "bytes": os.path.getsize(QCEW_ANNUAL_WIDE_OUTPUT_PATH),
        "hash": morpc.md5(QCEW_ANNUAL_WIDE_OUTPUT_PATH),
        "schema": ANNUAL_TABLE_SCHEMA_FILENAME,
        "profile":'tabular-data-resource'
    }
    
    # Create the resource object
    resource = frictionless.Resource(acsResource)

    print("Writing resource file to {}".format(QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH))
    cwd = os.getcwd()
    os.chdir(os.path.dirname(QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH))
    dummy = resource.to_yaml(os.path.basename(QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH))
    os.chdir(cwd)
    
    print("Validating resource on disk (including data and schema). This may take some time.")
    resourceOnDisk = frictionless.Resource(QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH)
    results = resourceOnDisk.validate()
    if(results.valid):
        print("Resource is valid\n")
    else:
        print("ERROR: Resource is NOT valid. Errors follow.\n")
        print(results)
        raise RuntimeError

Writing resource file to output_data\qcew_annual_wide_summarized.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid



## Create and validate resource file for quarterly wide-form table, if it exists

In [10]:
df_wide_quarterly = pd.read_csv(QCEW_QUARTERLY_WIDE_OUTPUT_PATH, dtype=quarterlyDtypes)

# Finding the maximum and minimum values in the 'year' column
max_year = df_wide_annual['year'].max()
min_year = df_wide_annual['year'].min()

# Update title and description with the county name
title = f"Compiled QCEW County Data, Quarterly, {min_year}-{max_year} (wide form)"
description = f"Employment and wage data for counties in MOPRC region, derived from the U.S. Bureau of Labor Statistics."

# Resource creation for WIDE QUARTERLY
if not df_wide_quarterly.empty:
    acsResource ={
    "name": "qcew_quarterly_wide",
    "title": title,
    "description": description,
    "path": QCEW_QUARTERLY_WIDE_OUTPUT_NAME,
    "format": "csv",
    "mediatype": "text/csv",
    "encoding": "utf-8",
    "bytes": os.path.getsize(QCEW_QUARTERLY_WIDE_OUTPUT_PATH),
    "hash": morpc.md5(QCEW_QUARTERLY_WIDE_OUTPUT_PATH),
    "schema": QUARTERLY_TABLE_SCHEMA_FILENAME,
    "profile":'tabular-data-resource'
    }
    
    # Create the resource object
    resource = frictionless.Resource(acsResource)

    print("Writing resource file to {}".format(QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE_PATH))
    cwd = os.getcwd()
    os.chdir(os.path.dirname(QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE_PATH))
    dummy = resource.to_yaml(os.path.basename(QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE_PATH))
    os.chdir(cwd)
    
    print("Validating resource on disk (including data and schema). This may take some time.")
    resourceOnDisk = frictionless.Resource(QCEW_QUARTERLY_WIDE_OUTPUT_RESOURCE_PATH)
    results = resourceOnDisk.validate()
    if(results.valid):
        print("Resource is valid\n")
    else:
        print("ERROR: Resource is NOT valid. Errors follow.\n")
        print(results)
        raise RuntimeError

Writing resource file to output_data\qcew_quarterly_wide_summarized.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid



## Create and validate resource file for annual long-form table, if it exists

In [11]:
df_wide_annual = pd.read_csv(QCEW_ANNUAL_LONG_OUTPUT_PATH, dtype=annualDtypes)

# Finding the maximum and minimum values in the 'year' column
max_year = df_wide_annual['year'].max()
min_year = df_wide_annual['year'].min()

# Update title and description with the county name
title = f"Compiled QCEW County Data, Annual, {min_year}-{max_year} (long form)"
description = f"Employment and wage data for counties in MOPRC region, derived from the U.S. Bureau of Labor Statistics."

# Resource creation for LONG ANNUAL
if not df_wide_annual.empty:
    acsResource = {
        "name": "qcew_annual_long",
        "title": title,
        "description": description,
        "path": QCEW_ANNUAL_LONG_OUTPUT_NAME,
        "format": "csv",
        "mediatype": "text/csv",
        "encoding": "utf-8",
        "bytes": os.path.getsize(QCEW_ANNUAL_LONG_OUTPUT_PATH),
        "hash": morpc.md5(QCEW_ANNUAL_LONG_OUTPUT_PATH),
        "schema": LONG_TABLE_SCHEMA_FILENAME,
        "profile":'tabular-data-resource'
    }
    
    # Create the resource object
    resource = frictionless.Resource(acsResource)

    print("Writing resource file to {}".format(QCEW_ANNUAL_LONG_OUTPUT_RESOURCE_PATH))
    cwd = os.getcwd()
    os.chdir(os.path.dirname(QCEW_ANNUAL_WIDE_OUTPUT_RESOURCE_PATH))
    dummy = resource.to_yaml(os.path.basename(QCEW_ANNUAL_LONG_OUTPUT_RESOURCE_PATH))
    os.chdir(cwd)
    
    print("Validating resource on disk (including data and schema). This may take some time.")
    resourceOnDisk = frictionless.Resource(QCEW_ANNUAL_LONG_OUTPUT_RESOURCE_PATH)
    results = resourceOnDisk.validate()
    if(results.valid):
        print("Resource is valid\n")
    else:
        print("ERROR: Resource is NOT valid. Errors follow.\n")
        print(results)
        raise RuntimeError

Writing resource file to output_data\qcew_annual_long_summarized.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid



## Create and validate resource file for quarterly long-form table, if it exists

In [ ]:
df_wide_annual = pd.read_csv(QCEW_QUARTERLY_LONG_OUTPUT_PATH, dtype=annualDtypes)

# Finding the maximum and minimum values in the 'year' column
max_year = df_wide_annual['year'].max()
min_year = df_wide_annual['year'].min()

# Update title and description with the county name
title = f"Compiled QCEW County Data, Quarterly, {min_year}-{max_year} (long form)"
description = f"Employment and wage data for counties in MOPRC region, derived from the U.S. Bureau of Labor Statistics."

# Resource creation for WIDE ANNUAL
if not df_wide_annual.empty:
    
    acsResource = {
      "name": "qcew_quarterly_long",
      "title": title,
      "description": description,
      "path": QCEW_QUARTERLY_LONG_OUTPUT_NAME,
      "format": "csv",
      "mediatype": "text/csv",
      "encoding": "utf-8",
      "bytes": os.path.getsize(QCEW_QUARTERLY_LONG_OUTPUT_PATH),
      "hash": morpc.md5(QCEW_QUARTERLY_LONG_OUTPUT_PATH),
      "schema": LONG_TABLE_SCHEMA_FILENAME,
      "profile":'tabular-data-resource'
    }
    
    # Create the resource object
    resource = frictionless.Resource(acsResource)

    print("Writing resource file to {}".format(QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE_PATH))
    cwd = os.getcwd()
    os.chdir(os.path.dirname(QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE_PATH))
    dummy = resource.to_yaml(os.path.basename(QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE_PATH))
    os.chdir(cwd)
    
    print("Validating resource on disk (including data and schema). This may take some time.")
    resourceOnDisk = frictionless.Resource(QCEW_QUARTERLY_LONG_OUTPUT_RESOURCE_PATH)
    results = resourceOnDisk.validate()
    if(results.valid):
        print("Resource is valid\n")
    else:
        print("ERROR: Resource is NOT valid. Errors follow.\n")
        print(results)
        raise RuntimeError